In [ ]:
pip install pyqubo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 44.0 MB/s eta 0:00:00


In [ ]:
pip install dwave.system

In [ ]:
pip install timeout_decorator

In [ ]:
pip install memory_profiler

In [ ]:
pip install python-tsp

In [ ]:
## Each variable in the hamiltonian is a test case. and the chain of test cases is the product of those variables

Setting up QUBO

In [ ]:
## First example.

from pyqubo import Spin
s1, s2 = Spin("s1"), Spin("s2")
H_test = 1*s1*s1 + 2*s1*s2 + 10*s2*s2

In [ ]:
print(H_test)

(((10.000000 * Spin('s2')) * Spin('s2')) + ((1.000000 * Spin('s1')) * Spin('s1')) + ((2.000000 * Spin('s1')) * Spin('s2')))


In [ ]:
from pyqubo import Binary
model = H_test.compile()
bqm = model.to_bqm()

In [ ]:
import neal
for i in range(0,10):
  sa = neal.SimulatedAnnealingSampler()
  sampleset = sa.sample(bqm, num_reads=10)
  decoded_samples = model.decode_sampleset(sampleset)
  best_sample = min(decoded_samples, key=lambda x: x.energy)
  print(best_sample.sample)

{'s2': 1, 's1': 0}
{'s2': 1, 's1': 0}
{'s2': 1, 's1': 0}
{'s2': 1, 's1': 0}
{'s2': 1, 's1': 0}
{'s2': 1, 's1': 0}
{'s2': 1, 's1': 0}
{'s2': 1, 's1': 0}
{'s2': 1, 's1': 0}
{'s2': 1, 's1': 0}


In [ ]:
## Define Arrays to store hamiltonian and results

model = []
H = []
qubo = []
offset = []

In [ ]:
a = (i**2+2)
b = (i**3+1)
c = i

In [ ]:
for i in range(0,100):
  hamiltonian = ((i**2+2)*s1 + (i**3+1)*s2 + i*s3 + s4)**2
  H.append(hamiltonian)
  #model_compile = hamiltonian.compile()
  #model.append(model_compile)
  print('hamiltonian = '+ str(hamiltonian))

In [ ]:
for i in range(0,len(H)):
  model_compile = H[i].compile()
  qubo.append(model_compile.to_qubo()[0])
  offset.append(model_compile.to_qubo()[1])

In [ ]:
print(len(qubo))

100


In [ ]:
for i in range(0,len(qubo)):
  print(qubo[i]) # doctest: +SKIP

Solving QUBO

In [ ]:
from pyqubo import Binary
x1, x2, x3, x4, x5 = Binary('x1'), Binary('x2'), Binary('x3'), Binary('x4'), Binary('x5')
model = H_test.compile()
bqm = model.to_bqm()

In [ ]:
import neal

sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=10)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.sample)

{'s5': 1, 's4': 1, 's3': 1, 's2': 0, 's1': 0}


In [ ]:
from pyqubo import Array
x = Array.create('x', shape=(2, 3), vartype='BINARY')
x[0, 1] + x[1, 2]


(Binary('x[0][1]') + Binary('x[1][2]'))

In [ ]:
from pyqubo import Array
numbers = [4, 2, 7, 1]
s = Array.create('s', shape=4, vartype='SPIN')
H = sum(n * s for s, n in zip(s, numbers))**2
model = H.compile()
qubo, offset = model.to_qubo()
print(qubo)

{('s[2]', 's[2]'): -196.0, ('s[0]', 's[0]'): -160.0, ('s[1]', 's[0]'): 64.0, ('s[2]', 's[1]'): 112.0, ('s[3]', 's[3]'): -52.0, ('s[2]', 's[0]'): 224.0, ('s[3]', 's[1]'): 16.0, ('s[3]', 's[0]'): 32.0, ('s[3]', 's[2]'): 56.0, ('s[1]', 's[1]'): -96.0}


In [ ]:
from pyqubo import Binary, Constraint



In [ ]:
pip install timeout_decorator

In [ ]:
pip install memory_profiler

In [ ]:
from pyqubo import Array, Constraint, Placeholder
import logging
import time
import argparse
import numpy as np
from timeout_decorator import timeout, TimeoutError
from memory_profiler import memory_usage


parser = argparse.ArgumentParser()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("benchmark_tsp")


In [ ]:
def number_partition_with_timeout(n, timeout_sec):

    @timeout(timeout_sec)
    def number_partition(n):
        t0 = time.time()
        s = Array.create('s', n, 'SPIN')
        numbers = np.random.randint(0, 10, n)
        H = sum([si * ni for si, ni in zip(s, numbers)])**2

        #Compile model
        t1 = time.time()
        model = H.compile()
        t2 = time.time()
        qubo, offset = model.to_qubo(index_label=False)
        t3 = time.time()
        print("len(qubo)", len(qubo))

        return t1-t0, t2-t1, t3-t2

    return number_partition(n)

In [ ]:
def tsp_with_timeout(n_city, timeout_sec):

    @timeout(timeout_sec)
    def tsp(n_city):
        t0 = time.time()
        x = Array.create('c', (n_city, n_city), 'BINARY')
        use_for_loop=False

        #Constraint not to visit more than two cities at the same time.
        time_const = 0.0
        for i in range(n_city):
            # If you wrap the hamiltonian by Const(...), this part is recognized as constraint
            time_const += Constraint((sum(x[i, j] for j in range(n_city)) - 1)**2, label="time{}".format(i))

        #Constraint not to visit the same city more than twice.
        city_const = 0.0
        for j in range(n_city):
            city_const += Constraint((sum(x[i, j] for i in range(n_city)) - 1)**2, label="city{}".format(j))

        #distance of route
        feed_dict = {}

        if use_for_loop:
            distance = 0.0
            for i in range(n_city):
                for j in range(n_city):
                    for k in range(n_city):
                        # we set the constant distance
                        d_ij = 10
                        distance += d_ij * x[k, i] * x[(k + 1) % n_city, j]

        else:
            distance = []
            for i in range(n_city):
                for j in range(n_city):
                    for k in range(n_city):
                        # we set the constant distance
                        d_ij = 10
                        distance.append(d_ij * x[k, i] * x[(k + 1) % n_city, j])
            distance = sum(distance)

        print("express done")

        #Construct hamiltonian
        A = Placeholder("A")
        H = distance

        feed_dict["A"] = 1.0

        #Compile model
        t1 = time.time()
        model = H.compile()
        t2 = time.time()
        qubo, offset = model.to_qubo(index_label=False, feed_dict=feed_dict)
        t3 = time.time()

        print("len(qubo)", len(qubo))

        return t1-t0, t2-t1, t3-t2

    return tsp(n_city)

In [ ]:
def measure(problem, step, init_size, max_size, timeout):
    if problem == "tsp":
        f = tsp_with_timeout
    elif problem == "number_partition":
        f = number_partition_with_timeout

    for n in range(init_size, max_size+step, step):
        try:
            max_memory, (express_time, compile_time, to_qubo_time) = memory_usage((f, (n, timeout)), max_usage=True, retval=True)
            logger.info("Memory usage is {} MB for n={}".format(max_memory, n))
            logger.info("Elapsed time is {} sec (expression: {} sec, compile: {} sec, to_qubo {} sec), for n={}".format(
                express_time+compile_time+to_qubo_time, express_time, compile_time, to_qubo_time, n))

        except TimeoutError as e:
            logger.error("TimeoutError: Elapsed time exceeded {} sec for n_city={}".format(timeout, n))
            break

In [ ]:
number_partition_with_timeout(1000, timeout_sec=10)


len(qubo) 407253


(0.015935897827148438, 0.9582388401031494, 0.8185317516326904)

In [ ]:
## Run TSP without QUBO

In [ ]:
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming
import time

distance_matrix = np.array([
 [0 ,5 ,4 ,8 ,8 ,8 ,3 ,3 ,1 ,9 ,4 ,1 ,4 ,4 ,4 ,8 ,3 ,0 ,3 ,2 ,9 ,1 ,2 ,8 ,9 ,3 ,2 ,7 ,8 ,2 ,9 ,2 ,9 ,1 ,5],
 [5 ,0 ,3 ,1 ,3 ,8 ,1 ,4 ,0 ,8 ,3 ,3 ,0 ,2 ,1 ,1 ,5 ,3 ,1 ,0 ,3 ,7 ,9 ,0 ,4 ,2 ,0 ,9 ,1 ,7 ,1 ,4 ,6 ,0 ,3],
 [4 ,3 ,0 ,4 ,7 ,6 ,5 ,4 ,9 ,4 ,6 ,3 ,2 ,4 ,2 ,3 ,5 ,6 ,5 ,6 ,0 ,4 ,9 ,3 ,2 ,2 ,7 ,7 ,7 ,9 ,4 ,4 ,2 ,5 ,5],
 [8 ,1 ,4 ,0 ,7 ,5 ,1 ,3 ,8 ,5 ,1 ,9 ,4 ,9 ,2 ,5 ,0 ,9 ,3 ,7 ,1 ,0 ,6 ,4 ,0 ,1 ,4 ,9 ,6 ,2 ,1 ,3 ,8 ,6 ,3],
 [8 ,3 ,7 ,7 ,0 ,8 ,5 ,0 ,2 ,3 ,2 ,8 ,9 ,4 ,7 ,5 ,6 ,0 ,8 ,4 ,2 ,5 ,8 ,6 ,3 ,3 ,5 ,4 ,1 ,3 ,8 ,2 ,7 ,0 ,6],
 [8 ,8 ,6 ,5 ,8 ,0 ,7 ,8 ,1 ,8 ,0 ,5 ,6 ,2 ,5 ,4 ,1 ,5 ,2 ,0 ,9 ,1 ,6 ,6 ,9 ,9 ,9 ,5 ,7 ,0 ,2 ,0 ,1 ,2 ,3],
 [3 ,1 ,5 ,1 ,5 ,7 ,0 ,6 ,7 ,6 ,6 ,9 ,4 ,4 ,5 ,2 ,5 ,6 ,7 ,3 ,4 ,4 ,1 ,5 ,3 ,1 ,8 ,8 ,4 ,6 ,8 ,7 ,4 ,6 ,8],
 [3 ,4 ,4 ,3 ,0 ,8 ,6 ,0 ,9 ,0 ,2 ,6 ,4 ,0 ,5 ,9 ,4 ,1 ,1 ,0 ,9 ,0 ,3 ,4 ,7 ,5 ,6 ,1 ,7 ,8 ,0 ,3 ,1 ,3 ,7],
 [1 ,0 ,9 ,8 ,2 ,1 ,7 ,9 ,0 ,1 ,2 ,7 ,2 ,6 ,1 ,3 ,4 ,3 ,8 ,4 ,7 ,6 ,0 ,4 ,0 ,6 ,6 ,6 ,0 ,0 ,2 ,6 ,1 ,9 ,1],
 [9 ,8 ,4 ,5 ,3 ,8 ,6 ,0 ,1 ,0 ,2 ,4 ,1 ,5 ,1 ,4 ,7 ,0 ,2 ,8 ,0 ,8 ,0 ,7 ,0 ,2 ,6 ,3 ,0 ,8 ,1 ,9 ,6 ,0 ,2],
 [4 ,3 ,6 ,1 ,2 ,0 ,6 ,2 ,2 ,2 ,0 ,0 ,7 ,0 ,3 ,0 ,5 ,1 ,8 ,2 ,8 ,2 ,6 ,7 ,2 ,9 ,6 ,4 ,8 ,0 ,4 ,2 ,6 ,1 ,4],
 [1 ,3 ,3 ,9 ,8 ,5 ,9 ,6 ,7 ,4 ,0 ,0 ,4 ,3 ,0 ,8 ,5 ,6 ,3 ,5 ,2 ,9 ,2 ,9 ,3 ,2 ,6 ,5 ,3 ,0 ,8 ,2 ,1 ,6 ,1],
 [4 ,0 ,2 ,4 ,9 ,6 ,4 ,4 ,2 ,1 ,7 ,4 ,0 ,3 ,4 ,9 ,0 ,9 ,7 ,8 ,1 ,3 ,2 ,5 ,4 ,5 ,8 ,8 ,9 ,3 ,0 ,6 ,2 ,0 ,2],
 [4 ,2 ,4 ,9 ,4 ,2 ,4 ,0 ,6 ,5 ,0 ,3 ,3 ,0 ,8 ,5 ,3 ,0 ,6 ,7 ,6 ,5 ,5 ,8 ,9 ,9 ,5 ,5 ,2 ,7 ,4 ,6 ,4 ,4 ,5],
 [4 ,1 ,2 ,2 ,7 ,5 ,5 ,5 ,1 ,1 ,3 ,0 ,4 ,8 ,0 ,4 ,6 ,1 ,0 ,3 ,5 ,9 ,4 ,1 ,8 ,8 ,7 ,5 ,2 ,9 ,1 ,8 ,9 ,3 ,2],
 [8 ,1 ,3 ,5 ,5 ,4 ,2 ,9 ,3 ,4 ,0 ,8 ,9 ,5 ,4 ,0 ,9 ,7 ,6 ,3 ,4 ,8 ,5 ,9 ,8 ,8 ,1 ,4 ,7 ,2 ,6 ,6 ,6 ,3 ,8],
 [3 ,5 ,5 ,0 ,6 ,1 ,5 ,4 ,4 ,7 ,5 ,5 ,0 ,3 ,6 ,9 ,0 ,2 ,4 ,8 ,5 ,3 ,5 ,4 ,9 ,2 ,0 ,0 ,6 ,3 ,4 ,4 ,5 ,1 ,8],
 [0 ,3 ,6 ,9 ,0 ,5 ,6 ,1 ,3 ,0 ,1 ,6 ,9 ,0 ,1 ,7 ,2 ,0 ,7 ,7 ,8 ,2 ,9 ,9 ,8 ,8 ,0 ,8 ,9 ,2 ,5 ,2 ,5 ,3 ,5],
 [3 ,1 ,5 ,3 ,8 ,2 ,7 ,1 ,8 ,2 ,8 ,3 ,7 ,6 ,0 ,6 ,4 ,7 ,0 ,2 ,8 ,1 ,0 ,8 ,4 ,3 ,4 ,1 ,7 ,0 ,4 ,6 ,7 ,8 ,3],
 [2 ,0 ,6 ,7 ,4 ,0 ,3 ,0 ,4 ,8 ,2 ,5 ,8 ,7 ,3 ,3 ,8 ,7 ,2 ,0 ,7 ,1 ,9 ,4 ,0 ,5 ,1 ,5 ,7 ,7 ,7 ,2 ,2 ,5 ,5],
 [9 ,3 ,0 ,1 ,2 ,9 ,4 ,9 ,7 ,0 ,8 ,2 ,1 ,6 ,5 ,4 ,5 ,8 ,8 ,7 ,0 ,7 ,9 ,4 ,8 ,7 ,7 ,2 ,9 ,8 ,8 ,3 ,7 ,0 ,5],
 [1 ,7 ,4 ,0 ,5 ,1 ,4 ,0 ,6 ,8 ,2 ,9 ,3 ,5 ,9 ,8 ,3 ,2 ,1 ,1 ,7 ,0 ,7 ,0 ,4 ,5 ,4 ,2 ,3 ,2 ,0 ,6 ,8 ,8 ,1],
 [2 ,9 ,9 ,6 ,8 ,6 ,1 ,3 ,0 ,0 ,6 ,2 ,2 ,5 ,4 ,5 ,5 ,9 ,0 ,9 ,9 ,7 ,0 ,1 ,4 ,5 ,6 ,8 ,6 ,1 ,6 ,0 ,3 ,7 ,2],
 [8 ,0 ,3 ,4 ,6 ,6 ,5 ,4 ,4 ,7 ,7 ,9 ,5 ,8 ,1 ,9 ,4 ,9 ,8 ,4 ,4 ,0 ,1 ,0 ,2 ,7 ,3 ,8 ,1 ,3 ,1 ,9 ,5 ,6 ,2],
 [9 ,4 ,2 ,0 ,3 ,9 ,3 ,7 ,0 ,0 ,2 ,3 ,4 ,9 ,8 ,8 ,9 ,8 ,4 ,0 ,8 ,4 ,4 ,2 ,0 ,4 ,6 ,1 ,4 ,3 ,5 ,4 ,9 ,8 ,8],
 [3 ,2 ,2 ,1 ,3 ,9 ,1 ,5 ,6 ,2 ,9 ,2 ,5 ,9 ,8 ,8 ,2 ,8 ,3 ,5 ,7 ,5 ,5 ,7 ,4 ,0 ,8 ,8 ,5 ,2 ,0 ,3 ,4 ,5 ,5],
 [2 ,0 ,7 ,4 ,5 ,9 ,8 ,6 ,6 ,6 ,6 ,6 ,8 ,5 ,7 ,1 ,0 ,0 ,4 ,1 ,7 ,4 ,6 ,3 ,6 ,8 ,0 ,6 ,1 ,4 ,5 ,5 ,9 ,4 ,3],
 [7 ,9 ,7 ,9 ,4 ,5 ,8 ,1 ,6 ,3 ,4 ,5 ,8 ,5 ,5 ,4 ,0 ,8 ,1 ,5 ,2 ,2 ,8 ,8 ,1 ,8 ,6 ,0 ,1 ,4 ,0 ,0 ,8 ,4 ,2],
 [8 ,1 ,7 ,6 ,1 ,7 ,4 ,7 ,0 ,0 ,8 ,3 ,9 ,2 ,2 ,7 ,6 ,9 ,7 ,7 ,9 ,3 ,6 ,1 ,4 ,5 ,1 ,1 ,0 ,0 ,2 ,8 ,0 ,6 ,5],
 [2 ,7 ,9 ,2 ,3 ,0 ,6 ,8 ,0 ,8 ,0 ,0 ,3 ,7 ,9 ,2 ,3 ,2 ,0 ,7 ,8 ,2 ,1 ,3 ,3 ,2 ,4 ,4 ,0 ,0 ,2 ,8 ,1 ,1 ,1],
 [9 ,1 ,4 ,1 ,8 ,2 ,8 ,0 ,2 ,1 ,4 ,8 ,0 ,4 ,1 ,6 ,4 ,5 ,4 ,7 ,8 ,0 ,6 ,1 ,5 ,0 ,5 ,0 ,2 ,2 ,0 ,3 ,5 ,6 ,0],
 [2 ,4 ,4 ,3 ,2 ,0 ,7 ,3 ,6 ,9 ,2 ,2 ,6 ,6 ,8 ,6 ,4 ,2 ,6 ,2 ,3 ,6 ,0 ,9 ,4 ,3 ,5 ,0 ,8 ,8 ,3 ,0 ,7 ,0 ,2],
 [9 ,6 ,2 ,8 ,7 ,1 ,4 ,1 ,1 ,6 ,6 ,1 ,2 ,4 ,9 ,6 ,5 ,5 ,7 ,2 ,7 ,8 ,3 ,5 ,9 ,4 ,9 ,8 ,0 ,1 ,5 ,7 ,0 ,2 ,6],
 [1 ,0 ,5 ,6 ,0 ,2 ,6 ,3 ,9 ,0 ,1 ,6 ,0 ,4 ,3 ,3 ,1 ,3 ,8 ,5 ,0 ,8 ,7 ,6 ,8 ,5 ,4 ,4 ,6 ,1 ,6 ,0 ,2 ,0 ,9],
 [5 ,3 ,5 ,3 ,6 ,3 ,8 ,7 ,1 ,2 ,4 ,1 ,2 ,5 ,2 ,8 ,8 ,5 ,3 ,5 ,5 ,1 ,2 ,2 ,8 ,5 ,3 ,2 ,5 ,1 ,0 ,2 ,6 ,9 ,0],
])

In [ ]:
start_dp = time.time()
permutation, distance = solve_tsp_dynamic_programming(distance_matrix)
time_elapsed_dp = time.time() - start_dp

In [ ]:
distance

17

In [ ]:
time_elapsed_dp

In [ ]:
from python_tsp.heuristics import solve_tsp_simulated_annealing

In [ ]:
start_sa = time.time()
permutation, distance = solve_tsp_simulated_annealing(distance_matrix)
time_elapsed_sa = time.time() - start_sa

In [ ]:
permutation

[0,
 17,
 13,
 10,
 15,
 26,
 16,
 27,
 28,
 32,
 29,
 8,
 22,
 31,
 5,
 19,
 24,
 9,
 7,
 4,
 33,
 20,
 2,
 23,
 1,
 12,
 30,
 34,
 11,
 14,
 18,
 21,
 3,
 6,
 25]

In [ ]:
distance

13

In [ ]:
time_elapsed_sa

0.5431699752807617

In [ ]:
import time

def isSubsetSum(arr, n, sum):
	# Base Cases
	if sum == 0:
		return True
	if n == 0 and sum != 0:
		return False

	# If last element is greater than sum, then
	# ignore it
	if arr[n-1] > sum:
		return isSubsetSum(arr, n-1, sum)

	''' else, check if sum can be obtained by any of
	the following
	(a) including the last element
	(b) excluding the last element'''

	return isSubsetSum(arr, n-1, sum) or isSubsetSum(arr, n-1, sum-arr[n-1])

# Returns true if arr[] can be partitioned in two
# subsets of equal sum, otherwise false


def findPartion(arr, n):
	# Calculate sum of the elements in array
	sum = 0
	for i in range(0, n):
		sum += arr[i]
	# If sum is odd, there cannot be two subsets
	# with equal sum
	if sum % 2 != 0:
		return false

	# Find if there is subset with sum equal to
	# half of total sum
	return isSubsetSum(arr, n, sum // 2)


# Driver code
if __name__ == '__main__':
  arr = [9 ,1 ,4 ,1 ,8 ,2 ,8 ,0 ,2 ,1 ,4 ,8 ,0 ,4 ,1 ,6 ,4 ,5 ,4 ,7 ,8 ,0 ,6 ,1 ,5 ,0 ,5 ,0 ,2 ,2 ,0 ,3 ,5 ,6 ,0]
  n = len(arr)

# Function call
start_np = time.time()
if findPartion(arr, n) == True:
	print("Can be divided into two subsets of equal sum")
else:
	print("Can not be divided into two subsets of equal sum")
time_elapsed_np = time.time() - start_np


Can be divided into two subsets of equal sum


In [ ]:
time_elapsed_np

0.1002345085144043

In [ ]:
# Returns true if arr[] can be
# partitioned in two subsets of
# equal sum, otherwise false


def findPartition(arr, n):
	sum = 0
	i, j = 0, 0

	# calculate sum of all elements
	for i in range(n):
		sum += arr[i]

	if sum % 2 != 0:
		return false

	part = [[True for i in range(n + 1)]
			for j in range(sum // 2 + 1)]

	# initialize top row as true
	for i in range(0, n + 1):
		part[0][i] = True

	# initialize leftmost column,
	# except part[0][0], as 0
	for i in range(1, sum // 2 + 1):
		part[i][0] = False

	# fill the partition table in
	# bottom up manner
	for i in range(1, sum // 2 + 1):

		for j in range(1, n + 1):
			part[i][j] = part[i][j - 1]

			if i >= arr[j - 1]:
				part[i][j] = (part[i][j] or
							part[i - arr[j - 1]][j - 1])

	return part[sum // 2][n]


# Driver Code
arr = [9 ,1 ,4 ,1 ,8 ,2 ,8 ,0 ,2 ,1 ,4 ,8 ,0 ,4 ,1 ,6 ,4 ,5 ,4 ,7 ,8 ,0 ,6 ,1 ,5 ,0 ,5 ,0 ,2 ,2 ,0 ,3 ,5 ,6 ,0]
n = len(arr)

# Function call
start_np_dp = time.time()
if findPartition(arr, n) == True:
	print("Can be divided into two",
		"subsets of equal sum")
else:
	print("Can not be divided into ",
		"two subsets of equal sum")
time_elapsed_np_dp = time.time() - start_np_dp




Can be divided into two subsets of equal sum


In [ ]:
time_elapsed_np_dp

0.0018284320831298828